# Road Rage: Finding the Ideal Speed Limit

### Assumptions
* Drivers want to go up to 120 km/hr.
* The average car is 5 meters long.
* Drivers want at least a number of meters equal to their speed in meters/second between them and the next car.
* Drivers will accelerate 2 m/s<sup>2</sup> up to their desired speed as long as they have room to do so.
* If another car is too close, drivers will match that car's speed until they have room again.
* If a driver would hit another car by continuing, they stop.
* Drivers will randomly (10% chance each second) slow by 2 m/s.
* This section of road is one lane going one way.
* Assume that drivers enter the road at the speed they left.
* Simulation starts with 30 cars per kilometer, evenly spaced.

## Normal Mode
We have a 1 kilometer section of road being built and do not know what the speed limit should be. This notebook simulates the 1 kilometer of road. Even though this road is not circular, the simulation treats it as such in order to generate a continuous flow of traffic.

In [17]:
import math
import matplotlib.pyplot as plt
import numpy as np
import random
from traffic_lib import *
%matplotlib inline

In [60]:
class Car:
    """
    Responsibilities:
    - Know speed (in m/s)
    - Know distance to driver ahead
    - Keep distance from driver ahead
    - Accelerate if possible
    - Match speed of those ahead if within safety zone
    - Stop if new location would result in crash (0 distance to car ahead)
    """
    
    def __init__(self, location, gap=28, speed_limit=33, start_speed=28):
        self.location = location
        self.gap = gap
        
        # speed limit is in km/hr = 1000 m / 3600 s = 1/3.6 m/s;
        self.desired_speed = speed_limit
        
        self.speed = start_speed
        self.size = 5
        self.advance = 0
        self.bumper = self.update_bumper()
            

    
    def __str__(self):
        return 'Car(location={},gap={},speed={})'.format(self.location, self.gap, self.speed)
    
    def __repr__(self):
        return self.__str__()
    
    def drive(self, car_ahead): # TODO: UPDATE GAP!
        self.advance = self.speed - car_ahead.speed

        if car_ahead.location - (car_ahead.size - 1) - self.location > self.location - car_ahead.location:
            self.gap = car_ahead.location - (car_ahead.size - 1) - self.location
        else:
            self.gap = car_ahead.location + 1000 - (car_ahead.size - 1) - self.location

        if self.stop(car_ahead):
            return self
        elif self.match_speed(car_ahead):
            return self
        elif self.random_slowdown(car_ahead):
            return self
        elif self.accelerate(car_ahead):
            return self

    def stop(self, car_ahead):
        if self.gap - self.advance <= 0:
            self.speed = 0
            self.location = car_ahead.bumper - 1
            if self.location < 0:
                self.location += 1000
            self.gap = 0
            return True
        
    def match_speed(self, car_ahead):
        if self.gap - self.advance < self.speed:
            self.speed = car_ahead.speed
            self.gap = self.speed
            return True
        
    def random_slowdown(self, car_ahead):
        if random.random() < 0.1 and self.speed > 2:
            self.speed -= 2
            self.gap = (self.advance + 2)
            return True
    
    def accelerate(self, car_ahead):
        if self.speed < self.desired_speed:
            self.speed += 2
            self.gap = (self.advance - 2)
            return True
        
    def update_bumper(self):
        if self.location >= 4:
            self.bumper = self.location - (self.size - 1)
        else:
            self.bumper = self.location - self.size + 1000
            # Note that to make the bumper location correct, we need to add 1 when adding 1000

In [61]:
class Road:
    """
    Responsibilities:
    - Hold length of road
    - Keep a list of vehicles on road
        - Initialize with number of cars
        - (1000 - sum(vehicle.size)) // len(self.vehicles)
    - Hold potential for slowdown
    
    Collaborators:
    - Car
    """
    def __init__(self, num_trucks=0):
        self.total_vehicle_space = ((30 - num_trucks) * 5) + (num_trucks * 25)
        self.initial_gap = (1000 - self.total_vehicle_space) // 30
        self.vehicles = [Car((4 + 33*n), self.initial_gap) for n in range(30 - num_trucks)]
        
        if num_trucks > 0:
            [self.vehicles.append(Truck() for _ in range(num_trucks))]

        self.vehicles[-1].gap = (1000 - self.vehicles[-1].location)
        
    def reinsert_car(self):
        if self.vehicles[-1].location < self.total_vehicle_space:
            self.vehicles.insert(0, Car(self.vehicles.pop(-1).location))
            return True

In [62]:
class HighwaySim:
    """
    Responsibilities:
    - Have road place cars at beginning of road when they reach the end
    - Keep track of time (seconds)
    - Step through time (ticks)
        - For each car on Road, tell car behind new situation and allow it to react
        - Update locations; location - 1000 if > 1000
    - Report stats; return traffic jam status
    - Pop, Append
    
    Collaborators:
    - Car
    - Road
    """
    
    def __init__(self):
        self.road = Road()
        self.ticks = 0
        self.is_traffic = []
    
    def iterate(self):
        """
        Reverse list to process cars at the end of the road first.
        n.b., lower numbers are physically ahead of higher numbers.
        """
        for idx, v in enumerate(self.road.vehicles):
            v.drive(self.road.vehicles[29 - idx]) # Hard coding number of cars
            
            if v.location + v.speed < 1000:
                v.location = v.location + v.speed
                v.bumper = v.update_bumper()
            else:
                v.location = v.location + v.speed - 1000
                self.road.reinsert_car()
            
            if v.speed == 0:
                self.is_traffic.append(True)
            else:
                self.is_traffic.append(False)
    
    def run_sim(self, duration=2):
        while self.ticks < duration:
            self.iterate()
            self.ticks += 1
        if self.is_traffic.count(True) > 0:
            return True
        else:
            return False

In [63]:
sim1 = HighwaySim()

In [64]:
sim1.run_sim()

False

In [65]:
sim1.is_traffic.count('True')

0

In [66]:
sim1.road.vehicles

[Car(location=15,gap=28,speed=28),
 Car(location=66,gap=2,speed=32),
 Car(location=95,gap=2,speed=28),
 Car(location=132,gap=-2,speed=32),
 Car(location=165,gap=-2,speed=32),
 Car(location=198,gap=-2,speed=32),
 Car(location=231,gap=-2,speed=32),
 Car(location=264,gap=-2,speed=32),
 Car(location=297,gap=-2,speed=32),
 Car(location=330,gap=-2,speed=32),
 Car(location=363,gap=2,speed=32),
 Car(location=388,gap=-6,speed=28),
 Car(location=429,gap=-2,speed=32),
 Car(location=462,gap=-2,speed=32),
 Car(location=495,gap=-2,speed=32),
 Car(location=520,gap=-6,speed=28),
 Car(location=561,gap=0,speed=32),
 Car(location=594,gap=-4,speed=32),
 Car(location=627,gap=-4,speed=32),
 Car(location=660,gap=-4,speed=32),
 Car(location=693,gap=0,speed=32),
 Car(location=718,gap=-8,speed=28),
 Car(location=759,gap=-4,speed=32),
 Car(location=792,gap=-4,speed=32),
 Car(location=825,gap=-4,speed=32),
 Car(location=858,gap=-4,speed=32),
 Car(location=891,gap=-4,speed=32),
 Car(location=924,gap=-4,speed=32),


In [38]:
def highway_trials(num_trials=1000):
    """
    Run num_trials of run_sim
    """
    trial_stats = []
    for _ in range(num_trials):
        trial_data = HighwaySim()
        trial_stats.append(trial_data)
    return trial_stats

In [9]:
highway_1000_data = highway_trials()

In [10]:
print(highway_1000_data.count(True))

0
